# Referenced Mosaic

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, mosaic, raster

## Settings

In [ ]:
settings = {
    'referenced_image_dir': '/Users/Shared/data/nitelite/images/220513-FH135/referenced',
    'mosaic_filepath': '/Users/Shared/data/nitelite/mosaics/220513-FH135/referenced2.tiff',
    'overwrite': True,
    'random_state': 16849,
}

## Parse Settings

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_image_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

# Extract/Transform/Load

In [ ]:
preprocessing = Pipeline([
    ('geotiff', preprocess.GeoTIFFPreprocesser()),
    ('geobounds', preprocess.GeoBoundsPreprocesser(passthrough=['filepath'])),
])

In [ ]:
X = preprocessing.fit_transform(
    referenced_fps,
)

# Build Mosaic

In [ ]:
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
model = mosaic.ReferencedMosaic(settings['mosaic_filepath'])

In [ ]:
model.fit_transform(X)

# Evaluate

## Individual Inspection

In [ ]:
ind = random_state.choice(X.index)
row = X.loc[ind]

In [ ]:
original_image = raster.ReferencedImage.open(row['filepath'])
mosaic_img = model.get_image(row['x_min'], row['x_max'], row['y_min'], row['y_max'])
mosaic_image = raster.ReferencedImage(mosaic_img, [row['x_min'], row['x_max']], [row['y_min'], row['y_max']])

In [ ]:
subplot_mosaic = [['original', 'mosaic']]
n_rows = len(subplot_mosaic)
n_cols = len(subplot_mosaic[0])

fig = plt.figure(figsize=(10*n_cols, 10*n_rows))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['original']
original_image.show(ax, crs='cartesian')

ax = ax_dict['mosaic']
mosaic_image.show(ax, crs='cartesian')

## Score

In [ ]:
r_median = model.score(X)